## Obtain data

In [1]:
!wget https://d396qusza40orc.cloudfront.net/dataminingcapstone/YelpDataset/yelp_dataset.tar.gz
!tar -xvf /kaggle/working/yelp_dataset.tar.gz

--2024-03-22 06:48:13--  https://d396qusza40orc.cloudfront.net/dataminingcapstone/YelpDataset/yelp_dataset.tar.gz
Resolving d396qusza40orc.cloudfront.net (d396qusza40orc.cloudfront.net)... 13.35.159.60, 13.35.159.89, 13.35.159.211, ...
Connecting to d396qusza40orc.cloudfront.net (d396qusza40orc.cloudfront.net)|13.35.159.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 443445047 (423M) [application/x-gzip]
Saving to: 'yelp_dataset.tar.gz'

yelp_dataset.tar.gz 100%[===================>] 422.90M  18.9MB/s    in 24s     

2024-03-22 06:48:39 (17.7 MB/s) - 'yelp_dataset.tar.gz' saved [443445047/443445047]

yelp_dataset_challenge_academic_dataset/
yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_user.json
yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json
yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_checkin.json
yelp_dataset_challenge_academic_dataset/Dataset_Challenge_Academic_Dataset_Agreement.pdf
yelp_dataset

## Dependencies

In [2]:
!pip install gensim
# !pip install

In [13]:
from gensim import corpora, matutils
from gensim.models import LdaModel
import json
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
import numpy as np
from nltk.tokenize import word_tokenize

### preprocessing

In [4]:
np.random.seed(1875)

In [5]:
# transforming to df
REVIEW_SOURCE = "/kaggle/working/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json"
reviews_df = pd.read_json(REVIEW_SOURCE, lines=True)
reviews_part_df = reviews_df
reviews_part_df.head()

,votes,user_id,review_id,stars,date,text,type,business_id
0,"{'funny': 0, 'useful': 2, 'cool': 1}",Xqd0DzHaiyRqVH3WRG7hzg,15SdjuK7DmYqUAj6rjGowg,5,2007-05-17,dr. goldberg offers everything i look for in a...,review,vcNAWiLM4dR7D2nwwJ7nCA
1,"{'funny': 0, 'useful': 2, 'cool': 0}",H1kH6QZV7Le4zqTRNxoZow,RF6UnRTtG7tWMcrO2GEoAg,2,2010-03-22,"Unfortunately, the frustration of being Dr. Go...",review,vcNAWiLM4dR7D2nwwJ7nCA
2,"{'funny': 0, 'useful': 1, 'cool': 1}",zvJCcrpm2yOZrxKffwGQLA,-TsVN230RCkLYKBeLsuz7A,4,2012-02-14,Dr. Goldberg has been my doctor for years and ...,review,vcNAWiLM4dR7D2nwwJ7nCA
3,"{'funny': 0, 'useful': 0, 'cool': 0}",KBLW4wJA_fwoWmMhiHRVOA,dNocEAyUucjT371NNND41Q,4,2012-03-02,Been going to Dr. Goldberg for over 10 years. ...,review,vcNAWiLM4dR7D2nwwJ7nCA
4,"{'funny': 0, 'useful': 2, 'cool': 1}",zvJCcrpm2yOZrxKffwGQLA,ebcN2aqmNUuYNoyvQErgnA,4,2012-05-15,Got a letter in the mail last week that said D...,review,vcNAWiLM4dR7D2nwwJ7nCA


In [6]:
reviews_cn_df = reviews_df[reviews_df['text'].str.contains("Chinese|China", case=False)]
reviews_cn_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24386 entries, 51 to 1125062
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   votes        24386 non-null  object        
 1   user_id      24386 non-null  object        
 2   review_id    24386 non-null  object        
 3   stars        24386 non-null  int64         
 4   date         24386 non-null  datetime64[ns]
 5   text         24386 non-null  object        
 6   type         24386 non-null  object        
 7   business_id  24386 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 1.7+ MB


In [7]:
reviews_cn_lst = reviews_cn_df['text'].to_list()

## Tokenize texts

In [8]:
pip install --upgrade gensim

Note: you may need to restart the kernel to use updated packages.


In [9]:
from gensim.models.phrases import Phrases, Phraser

In [10]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [18]:
from nltk import pos_tag, RegexpParser, tree2conlltags
import itertools

In [23]:
# sentences = []

# for i in range(len(reviews_cn_lst)):
#     sentences.append(word_tokenize(reviews_cn_lst[i]))
   
nouns_sentences = []
# Load the English language model (download if necessary)
# nlp = spacy.load("en_core_web_sm")

# # Your text or tokens (replace with your actual data)
# for review in reviews_cn_lst:
#     tokens = nlp(review)
#     # Extract nouns based on POS tag
#     nouns = [token.text for token in tokens if (token.pos_ == "NOUN" or token.pos_ == "VERB" or token.pos_ == "ADJ")]
#     nouns_sentences.append(nouns)
# with NLTK to extract nouns phrases
for review in reviews_cn_lst:
    tokens = word_tokenize(review.lower())
    tagged_tokens = pos_tag(tokens)

    # Define chunk grammar for noun phrases
    grammar = "NP: {<JJ><NN|NNS>+}"

    # Parsing
    cp = RegexpParser(grammar)
    chunked = cp.parse(tagged_tokens)

    # Convert to IOB and extract noun phrases
    iob_tagged = tree2conlltags(chunked)
    noun_phrases = [" ".join(word for word, tag, chunk in group) for key, group in itertools.groupby(iob_tagged, lambda word_tag_chunk: word_tag_chunk[2] != 'O') if key]
    noun_tokens = []
    for p in noun_phrases:
        noun_tokens += p.split()
    
    nouns_sentences.append(noun_tokens)

In [24]:
# Creating Phrases object
phrases = Phrases(nouns_sentences, min_count=20, threshold=50)

# Creating Phraser object for efficiency
phraser = Phraser(phrases)
total_phrases = []

# Transform sentences and print phrases
for sentence in nouns_sentences:
    transformed_sentence = phraser[sentence]
    phrase_list = [phrase.replace("_", " ") for phrase in transformed_sentence if "_" in phrase]
    total_phrases.append(phrase_list)

In [25]:
flattened_list = [item for sublist in total_phrases for item in sublist]
unique_phrases = set(flattened_list)

In [26]:
unique_phrases

{'    ',
 '                    ',
 '                         ',
 '                                        ',
 ' crab',
 ' gpmevlaxt4xvmjgka r q',
 ' lujqta7ajhzgk8dx6f6aw',
 ' your ',
 '-  -',
 '//4.bp.blogspot.com/ idzsav6c7am/saa919cmlfi/aaaaaaaabcu/hjxo-9x5wuu/s400/pucca+front.jpg',
 '//blogs.laweekly.com/squidink/2012/06/david chan chinese restaurants.php',
 '//s3-media2.ak.yelpcdn.com/bphoto/x9epm n2hghqh-fqxaqfvq/l.jpg',
 '^    ^~',
 '^   ^',
 'above average',
 'added bonus',
 'al dente',
 'alaskan king',
 'bacchanal buffet',
 'bachi burger',
 'baked goods',
 'bamboo shoots',
 'banh mi',
 'bay area',
 'bean curd',
 'bean sprouts',
 'beaten path',
 'bell peppers',
 'below average',
 'big wong',
 'bitter melon',
 'black bean',
 'black beans',
 'black pepper',
 'bok choy',
 'bottled water',
 'bottom line',
 'brown gravy',
 'cakes ',
 'caramelized onions',
 'carne asada',
 'central phoenix',
 'ceo3s2d 3mfdlduvy29jsg',
 'char siu',
 'cheap eats',
 'chilean sea',
 'chili oil',
 'china 

In [30]:
def save_non_null_elements(data, filename):
    with open(filename, 'w') as f:
        for cuisine in data:
            f.write(cuisine + '\n')


save_non_null_elements(unique_phrases, 'new_cuisines.txt')